# Yahoo Finance
- [Stock Price OHLCV Candles](https://finnhub.io/docs/api/stock-candles)


In [1]:
using Pkg
Pkg.activate("../julia")
# include("/root/project/notebooks/DeePC.jl")

  Activating project at `~/project/julia`


In [8]:
import HTTP
import Logging
log=Logging

Logging

In [11]:
api_key=get(ENV,"FINNHUB_API_KEY",nothing)
max_attempts=get(ENV,"QUANDL_DOWNLOAD_ATTEMPTS",5)
# YAHOO_URL="http://finance.yahoo.com/d/quotes.csv"
symbol="AAPL"
# symbol="GOOG,AAPL"

# YAHOO_URL="https://query1.finance.yahoo.com/v7/finance/download/$symbol?"
# YAHOO_URL="https://query1.finance.yahoo.com/v8/finance/chart/$symbol?"
YAHOO_URL = "https://query1.finance.yahoo.com/v8/finance/chart/$symbol?"
urlencode(x) = HTTP.URIs.escapeuri(x)
params=Dict(
    "period1"=>"1577836800",
    "period2"=>"1580515200",
    "interval"=>"1d",
    "events"=>"div%7Csplit",
    "includePrePost "=>"true",
)
    # "period1"=>"2020-01-01",
    # "period2"=>"2020-02-01",

# interval - The time interval between two data points. Can be 1m 2m 5m 15m 30m 60m 90m 1h 1d 5d 1wk 1mo 3mo
# _min_date 
# params
url=YAHOO_URL*urlencode(params)
# "https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=2020-01-01&period"

@info url
r=HTTP.request("GET",url)
resp=deepcopy(r.body)
String(resp)
# show_progress=true
# api_key

[ Info: https://query1.finance.yahoo.com/v8/finance/chart/AAPL?events=div%257Csplit&interval=1d&period2=1580515200&includePrePost%20=true&period1=1577836800


"{\"chart\":{\"result\":[{\"meta\":{\"currency\":\"USD\",\"symbol\":\"AAPL\",\"exchangeName\":\"NMS\",\"instrumentType\":\"EQUITY\",\"firstTradeDate\":345479400,\"regularMarketTime\":1688155205,\"gmtoffset\":-14400,\"timezone\":\"EDT\",\"exchangeTimezoneName\":\"America/New_York\",\"regularMarketPrice\":193." ⋯ 2511 bytes ⋯ "7.40422821044922,76.35901641845703,76.03179168701172,76.98419952392578,77.83649444580078,77.30899047851562,77.58496856689453,77.95858764648438,77.73390197753906,75.44813537597656,77.5824966430664,79.20649719238281,79.09171295166016,75.58489990234375]}]}}],\"error\":null}}"

In [17]:
r=HTTP.request("GET",url)
resp=deepcopy(r.body)
String(r.body)
import JSON
import Dates
import DataFrames
resp_json = JSON.parse(String(resp))
resp_json["chart"]["result"][1]["meta"]#["validRanges"]
resp_json["chart"]["result"][1]["meta"]
resp_json["chart"]["result"][1]["indicators"]["quote"][1]
# resp_json["chart"]["result"][1]["indicators"]["adjclose"]
gmt_offset = resp_json["chart"]["result"][1]["meta"]["gmtoffset"]
unix_timestaps_vector = resp_json["chart"]["result"][1]["timestamp"]
date_vector= [Dates.unix2datetime(x+gmt_offset) for x in resp_json["chart"]["result"][1]["timestamp"]]
df = DataFrames.DataFrame(resp_json["chart"]["result"][1]["indicators"]["quote"][1])
df[:,:"date"]=date_vector
df[:,:"unix"]=unix_timestaps_vector
df[:,:"exchangeName"].=resp_json["chart"]["result"][1]["meta"]["exchangeName"]
df[:,:"timezone"].=resp_json["chart"]["result"][1]["meta"]["exchangeTimezoneName"]
df[:,:"currency"].=resp_json["chart"]["result"][1]["meta"]["currency"]
df[:,:"symbol"].=resp_json["chart"]["result"][1]["meta"]["symbol"]
df

Row,close,high,low,open,volume,date,unix,exchangeName,timezone,currency,symbol
,Any,Any,Any,Any,Any,DateTime,Any,String,String,String,String
1,75.0875,75.15,73.7975,74.06,135480400,2020-01-02T10:30:00,1577975400,NMS,America/New_York,USD,AAPL
2,74.3575,75.145,74.125,74.2875,146322800,2020-01-03T10:30:00,1578061800,NMS,America/New_York,USD,AAPL
3,74.95,74.99,73.1875,73.4475,118387200,2020-01-06T10:30:00,1578321000,NMS,America/New_York,USD,AAPL
4,74.5975,75.225,74.37,74.96,108872000,2020-01-07T10:30:00,1578407400,NMS,America/New_York,USD,AAPL
5,75.7975,76.11,74.29,74.29,132079200,2020-01-08T10:30:00,1578493800,NMS,America/New_York,USD,AAPL
6,77.4075,77.6075,76.55,76.81,170108400,2020-01-09T10:30:00,1578580200,NMS,America/New_York,USD,AAPL
7,77.5825,78.1675,77.0625,77.65,140644800,2020-01-10T10:30:00,1578666600,NMS,America/New_York,USD,AAPL
8,79.24,79.2675,77.7875,77.91,121532000,2020-01-13T10:30:00,1578925800,NMS,America/New_York,USD,AAPL
9,78.17,79.3925,78.0425,79.175,161954400,2020-01-14T10:30:00,1579012200,NMS,America/New_York,USD,AAPL


In [22]:
import DataFrames
import JSON
import HTTP
import Dates
function get_chart_data(symbol, period1, period2, freq)
    
    YAHOO_CHART_V8_URL="https://query1.finance.yahoo.com/v8/finance/chart/$symbol?"
    params=Dict(
        "period1"=>period1,
        "period2"=>period2,
        "interval"=>freq,
        "events"=>"div%7Csplit",
        "includePrePost "=>"true",
    )
    url=YAHOO_CHART_V8_URL *  HTTP.URIs.escapeuri(params)
    r=HTTP.request("GET",url)
    return r
end

function parse_intraday_raw_data(r)
    resp=deepcopy(r.body)
    resp_json = JSON.parse(String(resp))
    df = DataFrames.DataFrame(resp_json["chart"]["result"][1]["indicators"]["quote"][1])
    gmt_offset = resp_json["chart"]["result"][1]["meta"]["gmtoffset"]
    unix_timestaps_vector = resp_json["chart"]["result"][1]["timestamp"]
    date_vector= [Dates.unix2datetime(x+gmt_offset) for x in resp_json["chart"]["result"][1]["timestamp"]]
    df[:,:"date"]=date_vector
    df[:,:"unix"]=unix_timestaps_vector
    df[:,:"exchangeName"].=resp_json["chart"]["result"][1]["meta"]["exchangeName"]
    df[:,:"timezone"].=resp_json["chart"]["result"][1]["meta"]["exchangeTimezoneName"]
    df[:,:"currency"].=resp_json["chart"]["result"][1]["meta"]["currency"]
    df[:,:"symbol"].=resp_json["chart"]["result"][1]["meta"]["symbol"]
    return df
end

function get_interday_data(symbols, period1, period2;freq::String="5m")
    df=DataFrames.DataFrame()
    for ticker=symbols
        df = DataFrames.vcat(
             df,
             parse_intraday_raw_data(get_chart_data(ticker,period1,period2,freq))
            )   
    end
    return df
end

get_interday_data (generic function with 1 method)

In [23]:

period1="1577836800"
period2="1580515200"
tickers=["USDGBP=X"] # ForEx data
# tickers = ["AAPL","GOOG"] # Equity data
df =get_interday_data(tickers, period1, period2;freq="1d")


Row,close,high,low,open,volume,date,unix,exchangeName,timezone,currency,symbol
,Any,Any,Any,Any,Any,DateTime,Any,String,String,String,String
1,0.754,0.7565,0.753523,0.754489,0,2020-01-01T01:00:00,1577836800,CCY,Europe/London,GBP,GBP=X
2,0.7547,0.7619,0.7537,0.75469,0,2020-01-02T01:00:00,1577923200,CCY,Europe/London,GBP,GBP=X
3,0.7603,0.766,0.75981,0.76081,0,2020-01-03T01:00:00,1578009600,CCY,Europe/London,GBP,GBP=X
4,0.76452,0.76538,0.759071,0.76445,0,2020-01-06T01:00:00,1578268800,CCY,Europe/London,GBP,GBP=X
5,0.7593,0.76357,0.75688,0.75919,0,2020-01-07T01:00:00,1578355200,CCY,Europe/London,GBP,GBP=X
6,0.76256,0.76437,0.75933,0.7627,0,2020-01-08T01:00:00,1578441600,CCY,Europe/London,GBP,GBP=X
7,0.76306,0.76834,0.76193,0.76302,0,2020-01-09T01:00:00,1578528000,CCY,Europe/London,GBP,GBP=X
8,0.7651,0.76632,0.76361,0.76515,0,2020-01-10T01:00:00,1578614400,CCY,Europe/London,GBP,GBP=X
9,0.76654,0.77149,0.7665,0.76665,0,2020-01-13T01:00:00,1578873600,CCY,Europe/London,GBP,GBP=X


In [100]:
df=DataFrames.DataFrame()
for df_i=dfs
    df = DataFrames.vcat(df,df_i) # This works
end
df
# DataFrames.vcat([d for d in dfs])

# Cache

In [77]:
function get_cache_path()
    cache_path=get(ENV,"AIRBORNE_ROOT",nothing)
    if !(isnothing(cache_path))
        return cache_path
    elseif (Sys.islinux()) ||  (Sys.isapple()) 
        return "/root/tmp/.AirBorne/.cache"
    elseif Sys.iswindows() 
        return "$(ENV["HOME"])/.AirBorne/.cache"
    end
end

function gen_id()
   return Dates.format(Dates.now(Dates.UTC),"yyyy_mm_dd_H_M_S_s") 
end

function remove_bundle(bundle_id::String; just_archive::Bool=false)
    # Define directories
    cache_dir=get_cache_path()
    bundle_dir= joinpath(cache_dir,bundle_id)
    archive_dir= joinpath(bundle_dir,"archive")
    if (just_archive) && (isdir(archive_dir)) 
        rm(archive_dir; recursive=true)
    elseif !(just_archive) && (isdir(bundle_dir))
        rm(bundle_dir; recursive=true)
    end
end
function list_bundles()
    return readdir(get_cache_path(), sort=false)
end

list_bundles (generic function with 1 method)

In [72]:
import Dates
import Parquet2
function store_bundle(data::DataFrames.DataFrame; bundle_id::Union{Nothing, String}=nothing, archive::Bool=true, meta::Dict=Dict(), c_meta::Dict=Dict())
    # Define directories
    cache_dir=get_cache_path()
    bundle_id= !(isnothing(bundle_id)) ? bundle_id : gen_id()
    bundle_dir= joinpath(cache_dir,bundle_id)
    archive_dir= joinpath(bundle_dir,"archive")
    
    
    # Ensure directories exist
    if !(isdir(archive_dir))
       mkpath(archive_dir) 
    end

    if !(isdir(bundle_dir))
       mkpath(bundle_dir) 
    end

    # Move current contents to archive
    contents=readdir(bundle_dir)
    files_to_archive = [c for c in contents if c!="archive"]
    for files_to_archive=files_to_archive
        if archive
        mv(joinpath( bundle_dir,files_to_archive),joinpath(archive_dir,files_to_archive))
        else
        @info("Removing $(joinpath( bundle_dir,files_to_archive))")
        rm(joinpath( bundle_dir,files_to_archive))
        end
    end

    # Write file
    file_path = joinpath( bundle_dir, gen_id() * ".parq.snappy")
    
    @info("Storing $file_path")
    Parquet2.writefile(file_path, data;compression_codec=:snappy, metadata=meta, column_metadata=c_meta)
end

function load_bundle(bundle_id::String)
    cache_dir=get_cache_path()
    bundle_dir= joinpath(cache_dir,bundle_id)
    if !(isdir(bundle_dir))
      throw(error("Bundle does not exist.")) 
    end
    contents=readdir(bundle_dir)
    contents_to_read = [c for c in contents if c!="archive"]
    if size(contents_to_read)[1]!=1
        throw(error("$(size(contents_to_read)[1]) files found in bundle directory.")) 
    end
    file_path = joinpath(bundle_dir,contents_to_read[1])
    ds = Parquet2.Dataset(file_path) # Create a dataset
    df = DataFrames.DataFrame(ds; copycols=false) # Load from dataframed
    return df
end


load_bundle (generic function with 1 method)

In [73]:
# gen_id()
load_bundle("b2")

Row,close,high,low,open,volume,date,unix,exchangeName,timezone,currency,symbol
,Float64,Float64,Float64,Float64,Int64,DateTime,Int64,String,String,String,String
1,75.0875,75.15,73.7975,74.06,135480400,2020-01-02T10:30:00,1577975400,NMS,America/New_York,USD,AAPL
2,74.3575,75.145,74.125,74.2875,146322800,2020-01-03T10:30:00,1578061800,NMS,America/New_York,USD,AAPL
3,74.95,74.99,73.1875,73.4475,118387200,2020-01-06T10:30:00,1578321000,NMS,America/New_York,USD,AAPL
4,74.5975,75.225,74.37,74.96,108872000,2020-01-07T10:30:00,1578407400,NMS,America/New_York,USD,AAPL
5,75.7975,76.11,74.29,74.29,132079200,2020-01-08T10:30:00,1578493800,NMS,America/New_York,USD,AAPL
6,77.4075,77.6075,76.55,76.81,170108400,2020-01-09T10:30:00,1578580200,NMS,America/New_York,USD,AAPL
7,77.5825,78.1675,77.0625,77.65,140644800,2020-01-10T10:30:00,1578666600,NMS,America/New_York,USD,AAPL
8,79.24,79.2675,77.7875,77.91,121532000,2020-01-13T10:30:00,1578925800,NMS,America/New_York,USD,AAPL
9,78.17,79.3925,78.0425,79.175,161954400,2020-01-14T10:30:00,1579012200,NMS,America/New_York,USD,AAPL


In [67]:
bundle_id="b2"
data=df
meta=Dict("schema_id"=>"schema_1")
c_meta=Dict("close"=>Dict("description"=>"Close price")) # Add metadata 
ENV["AIRBORNE_ROOT"] = pwd() * "/.cache"

# store_bundle(data;c_meta=nothing)

# store_bundle(data;bundle_id=bundle_id,archive=true,meta=meta,c_meta=c_meta)
store_bundle(data;bundle_id=bundle_id,archive=true)


[ Info: Storing /root/project/notebooks/.cache/b2/2023_06_06_6_6_18_403.parq.snappy


✏ Parquet2.FileWriter{IOStream}(/root/project/notebooks/.cache/b2/2023_06_06_6_6_18_403.parq.snappy)

In [81]:
# list_bundles()
remove_bundle("b2";just_archive=true)

In [84]:
bundle_id="test_bundle"
ds = Parquet2.Dataset("/root/project/notebooks/demo.parq.snappy") # Create a dataset    
df = DataFrames.DataFrame(ds; copycols=false) # Load from dataframed
store_bundle(df;bundle_id=bundle_id,archive=true,meta=meta,c_meta=c_meta)

[ Info: Storing /root/project/notebooks/.cache/test_bundle/2023_06_06_8_13_59_224.parq.snappy


✏ Parquet2.FileWriter{IOStream}(/root/project/notebooks/.cache/test_bundle/2023_06_06_8_13_59_224.parq.snappy)

### Modules can be passed as arguments of Functions!

This is great as it means that I can pass an entity with associated attributes and methods to a function. 

In [85]:
module F
function  hi()
    print("hello")
end
end

Main.F

In [86]:
function h(x)
   x.hi() 
end

h (generic function with 1 method)

In [87]:
h(F)

hello

# Currencies

Retrieving currencies exchange rates using Yahoo Finance

In [2]:
using HTTP:HTTP
using Logging: @info


In [31]:
YAHOO_SPARK_V7_URL="https://query1.finance.yahoo.com/v7/finance/spark?"
period1="1577836800"
period2="1580515200"
# "period1"=>"2020-01-01",
# "period2"=>"2020-02-01",
urlencode(x) = HTTP.URIs.escapeuri(x)
freq="5m"
# symbols=USDGBP%3DX&start=1686610800&end=1686697140&gmtoffset=3600&dataGranularity=1d&range=1mo
params=Dict(
    # "start"=>period1,
    # "end"=>period2,
    "date"=>period1,
    # "period1"=>period1,
    # "period2"=>period2,
    # "interval"=>freq,
    "interval"=>freq,
    "symbols"=>"USDGBP=X",
    # "symbols"=>"USDGBP%3DX",
    # "includePrePost "=>"true",
)
url=YAHOO_SPARK_V7_URL *  HTTP.URIs.escapeuri(params)
@info url
r=HTTP.request("GET",url)
resp=deepcopy(r.body)
using DataFrames:DataFrame
using Dates: Dates
timestamps = resp_json["spark"]["result"][1]["response"][1]["timestamp"]
close=resp_json["spark"]["result"][1]["response"][1]["indicators"]["quote"][1]["close"]
gmt_offset = resp_json["spark"]["result"][1]["response"][1]["meta"]["gmtoffset"]
date_vector= [Dates.unix2datetime(x+gmt_offset) for x in timestamps]

DataFrame(Dict("unix"=>timestamps,"date"=>date_vector,"close"=>close))

[ Info: https://query1.finance.yahoo.com/v7/finance/spark?symbols=USDGBP%3DX&interval=5m&date=1577836800


Row,close,date,unix
,Any,DateTime,Any
1,0.8038,2023-06-08T00:00:00,1686178800
2,0.804,2023-06-08T00:05:00,1686179100
3,0.8039,2023-06-08T00:10:00,1686179400
4,0.8039,2023-06-08T00:15:00,1686179700
5,0.804,2023-06-08T00:20:00,1686180000
6,0.804,2023-06-08T00:25:00,1686180300
7,0.8038,2023-06-08T00:30:00,1686180600
8,0.8038,2023-06-08T00:35:00,1686180900
9,0.8037,2023-06-08T00:40:00,1686181200


In [5]:
using JSON: JSON
resp_json = JSON.parse(String(resp))

Dict{String, Any} with 1 entry:
  "spark" => Dict{String, Any}("error"=>nothing, "result"=>Any[Dict{String, Any…

### Modelling Currencies using JuliaFinance

https://github.com/JuliaFinance/Assets.jl
add Assets#master

In [16]:
using Currencies
using Assets: cash

LoadError: UndefVarError: cash not defined

In [39]:
macro money(syms)
    args = syms isa Expr ? syms.args : [syms] # If it's just one element then create a list of length one
    for nam in args
        @eval __module__ const $nam = Currencies.currency($nam) #Assets.$nam # Create functions
    end
end

@money (macro with 1 method)

In [84]:
# Julia program to illustrate the
# overloading of + operator

# Defining a custom type MyNumber
struct MyNumber
	value::Int
end


# Overload the + operator for MyNumber
Base.:+(x::MyNumber, y::MyNumber) = MyNumber(x.value + y.value)

# Creating MyNumber objects
a = MyNumber(3)
b = MyNumber(4)

# Adding both the objects
c = a + b
println(c)


MyNumber(7)


In [334]:
# cash(USD)
# @stock MSFT
# @money :GBP
# Assets.cash(:USD)
using Printf: @sprintf, format
using Currencies: unit
using Base: Base

"""Just a float with an associated Symbol"""
struct MoneyH{S}
	value::Float64
end


"""Just a wrapper around a dictionary"""
struct WalletB
	content::Dict{Symbol,Float64}
end

# WalletA(List[])

function WalletB(S::Symbol)
    WalletB(Dict(S=>0.0))
end

function WalletB(b::MoneyH{B}) where {B} 
    WalletB(Dict(B=>b.value))
end
# For Currencies

# For Wallets

get_symbol(::Type{MoneyH{S}}) where {S} = S # Symbol from Type
get_symbol(::MoneyH{S}) where {S} = S # Symbol from Instance 

Base.setindex!(w::WalletB,a::Float64, S::Symbol) = Base.setindex!(w.content,a,S)
Base.getindex(w::WalletB, S::Symbol) = Base.getindex(w.content,S)
Base.length(a::WalletB) = length(a.content)
Base.haskey(a::WalletB,key) = haskey(a.content,key)
Base.keys(a::WalletB) = keys(a.content)

Base.show(io::IO, ::MIME"text/plain", v::MoneyH{S}) where {S} = print(io, @sprintf("%.2f", v.value) ,"$S")
Base.show(io::IO, v::MoneyH{S}) where {S} = show(io,MIME("text/plain"),v)

function Base.show(io::IO, ::MIME"text/plain", w::WalletB) 
   println(io, "WalletB object containing:")
    for (key, value) in w.content
       println(io, "  ",@sprintf("%.2f", value),"$(key)" )
    end
end
Base.show(io::IO, w::WalletB) = show(io,MIME("text/plain"),w)


sameCurrencyB(a::MoneyH{S},b::MoneyH{D}) where {S,D} = S==D ? "Same" : "Different" 

Base.:+(a::MoneyH{A}, b::MoneyH{A}) where {A} = MoneyH{A}(a.value + b.value) # Same 
Base.:+(a::MoneyH{A}, b::MoneyH{B}) where {A,B} = WalletB(Dict(A=>a.value,B=>b.value)) #Different?

# Base.:+(a::MoneyH{A}, b::MoneyH{B}) where {A,B} = A==B ? MoneyH{A}(a.value + b.value) : WalletB(Dict(A=>a.value,B=>b.value))

Base.:*(a::Float64, c::Type{Currency{S}}) where {S} = MoneyH{S}(a) # allow 3.0USD as a valid expresion
Base.:*(c::Type{Currency{S}}, a::Float64) where {S} = MoneyH{S}(a) # allow USD*3.0 as a valid expresion
Base.:*(a::Real, c::Type{Currency{S}}) where {S} = MoneyH{S}(Float64(a)) # allow 3USD as a valid expresion
Base.:*(c::Type{Currency{S}},a::Real ) where {S} = MoneyH{S}(Float64(a)) # allow USD*3 as a valid expresion

Base.:*(a::Real, b::MoneyH{B}) where {B} = MoneyH{B}(a*b.value) # allow to multiply money by a value
Base.:*(b::MoneyH{B}, a::Real) where {B} = MoneyH{B}(a*b.value) # Commutability of product
# function addWalletB(a::WalletB, b::WalletB)
    
function Base.:+(a::WalletB, b::WalletB) 
    bigW, smallW =  length(a)>length(b) ? (a,b) : (b,a)
    out = deepcopy(bigW) # We don't want to modify original
    for (key, value) in smallW.content
        if haskey(out,key)
            out[key] += value
        else
            out[key] = value
        end
    end
    return out
end

Base.:+(a::WalletB, b::MoneyH) = a + WalletB(b) # Add money to the wallet just by summing
Base.:+(b::MoneyH, a::WalletB) = a + WalletB(b) # Commutability of operator

In [326]:

function exchange(a::MoneyH{A},B::Symbol, exchangeRate::Dict{Tuple{Symbol, Symbol}, Float64}) where {A}
    if haskey(exchangeRate,(A,B))
        return MoneyH{B}(a.value*exchangeRate[(A,B)])
    elseif haskey(exchangeRate,(B,A))
        return MoneyH{B}(a.value/exchangeRate[(B,A)])
    elseif B==A
        return a
    else
        throw( KeyError("($A,$B) or ($B,$A)"))
    end
end

exchange (generic function with 1 method)

In [341]:
println(a)
println(b)
println(a+a)
println(a+b)

31.00GBP
32.00USD
62.00GBP
WalletB object containing:
  31.00GBP
  32.00USD



In [300]:
c2 = WalletB(b)
d2 = WalletB(:USD)
d2[:DEMO]=3.0
println(c2)
println(d2)
r = c2+d2

print(r + 3GBP)
r+=5GBP
x=3GBP
x+= 5GBP*4
x
length(r)
# typeof(c2)
# d2
# length(d2.content)

WalletB(Dict(:USD => 32.0))
WalletB(Dict(:USD => 0.0, :DEMO => 3.0))
WalletB(Dict(:GBP => 3.0, :USD => 32.0, :DEMO => 3.0))

23.00GBP


In [333]:
exchangeRateI = Dict{Tuple{Symbol, Symbol}, Float64}(
    (:USD,:GBP)=>1.2,
    (:GBP,:USD)=>0.6,
    (:USD,:UYU)=>50,
)

println(a)
println(exchange(a,:USD,exchangeRateI))
println(exchange(a,:GBP,exchangeRateI))
exchange(a,:UYU,exchangeRateI)

31.00GBP
18.60USD
31.00GBP


LoadError: KeyError: key "(GBP,UYU) or (UYU,GBP)" not found

In [304]:
USDa = Currencies.currency(:USD)
GBP = Currencies.currency(:GBP)
# USD
3.0USDa
2USDa
USDa*3
# USDa

3.00USD


In [250]:
# sameCurrencyB(b,b)
# sameCurrencyB(b,a)
b+b
b+a
# b*b #Error because product of money is not defined-> Good!
3b

96.00USD


In [170]:
println("%.$(unit(:USD))f",b.value)
# unit(:USD)

%.2f32.0


In [187]:
# a = Currencies.currency(:USD)
# true || error("Currency $a is not defined.")
# typeof(a)
# typeof(Int) # DataType
# a(3.0)
# a
b=MoneyH{:USD}(32.0)
a=MoneyH{:GBP}(31.0)
typeof(b) 
# MoneyD(32)
 # MoneyD{}(1.0)
# f{T<:Integer}(::Type{T}) = "integer method, got $T"

MoneyH{:USD}

In [144]:

# findparam(ex::Example{T, N}) where {T, N} = N
# get_symbol(b)
# get_symbol typeof(b).parameters[1]

get_symbol (generic function with 3 methods)

In [155]:
Currencies.unit(:USD) # Decimal places when displaying
Currencies.name(:USD) # Decimal places when displaying
# Currencies.symbol(:USD) # Decimal places when displaying
"$(:USD)"


"USD"

In [146]:
get_symbol(b)

:USD

In [80]:
# Currencies.name(:($a)) # This calls a dictionary indexed by symbols with tuple values, fetches the symbol of the type of a and retrieves the 4th  
# element that happens to be the name
# Currencies.code(a) # Same but is the 3rd element

a = Currencies.currency(:USD)
println("$(a): $(typeof(a))")
println("$(typeof(Currency))")
println(" $(typeof(3))")
a <: Currency # true
a == a
# BABA=3
# Currency(:avsa)
Currencies.name(:UYU) # Accepts both raw symbols and currencies

get_symbol(::Type{Currency{S}}) where {S} = S # Retrieves symbol, function operates at type level


Currency{:USD}: DataType
UnionAll
 Int64


get_symbol (generic function with 1 method)

In [98]:
float{:USD}(2)

LoadError: TypeError: in Type{...} expression, expected UnionAll, got a value of type typeof(float)

# Using AirBorne:


### Retrieving Data

In [4]:
using AirBorne: AirBorne
using AirBorne.ETL.Cache: load_bundle, store_bundle
using Dates: Dates
cache_dir = joinpath(@__DIR__, "assets", "cache")
# To generate this data use:
from = Dates.DateTime("2017-01-01"); to = Dates.DateTime("2022-01-01")
u_from = string(round(Int, Dates.datetime2unix(from))); u_to = string(round(Int, Dates.datetime2unix(to)))
data = AirBorne.ETL.YFinance.get_interday_data(["AAPL","GOOG"], u_from, u_to)

# For some reason  the column type is still "Any". Minor bug but annoying. 
data[!,:close]=float.(data[!,:close])
data[!,:high]=float.(data[!,:high])
data[!,:low]=float.(data[!,:low])
data[!,:open]=float.(data[!,:open])
data[!,:unix]=Int.(data[!,:unix])
data[!,:volume]=Int.(data[!,:volume])
# show(data)
AirBorne.ETL.Cache.store_bundle(data; bundle_id="demo", archive=true) 
data2 = load_bundle("demo")

[ Info: Storing /root/project/.AirBorneCache/demo/2023_07_03_11_29_2_972.parq.snappy


Row,close,high,low,open,volume,date,unix,exchangeName,timezone,currency,symbol
,Float64,Float64,Float64,Float64,Int64,DateTime,Int64,String,String,String,String
1,29.0375,29.0825,28.69,28.95,115127600,2017-01-03T10:30:00,1483453800,NMS,America/New_York,USD,AAPL
2,29.005,29.1275,28.9375,28.9625,84472400,2017-01-04T10:30:00,1483540200,NMS,America/New_York,USD,AAPL
3,29.1525,29.215,28.9525,28.98,88774400,2017-01-05T10:30:00,1483626600,NMS,America/New_York,USD,AAPL
4,29.4775,29.54,29.1175,29.195,127007600,2017-01-06T10:30:00,1483713000,NMS,America/New_York,USD,AAPL
5,29.7475,29.8575,29.485,29.4875,134247600,2017-01-09T10:30:00,1483972200,NMS,America/New_York,USD,AAPL
6,29.7775,29.845,29.575,29.6925,97848400,2017-01-10T10:30:00,1484058600,NMS,America/New_York,USD,AAPL
7,29.9375,29.9825,29.65,29.685,110354400,2017-01-11T10:30:00,1484145000,NMS,America/New_York,USD,AAPL
8,29.8125,29.825,29.5525,29.725,108344800,2017-01-12T10:30:00,1484231400,NMS,America/New_York,USD,AAPL
9,29.76,29.905,29.7025,29.7775,104447600,2017-01-13T10:30:00,1484317800,NMS,America/New_York,USD,AAPL


### Currencies and Wallets

In [5]:
using AirBorne: AirBorne, Money, Wallet
using Currencies: Currencies
USD = Currencies.currency(:USD)
GBP = Currencies.currency(:GBP)
UYU = Currencies.currency(:UYU)
a = 10USD
b = 10.0USD
c1 = Money{:GBP}(5.0)
c2 = Money{:GBP}(5)
exchangeRateI = Dict{Tuple{Symbol,Symbol},Real}(
    (:USD, :GBP) => 2, (:GBP, :USD) => 1//2, (:USD, :UYU) => 50
)

AirBorne.exchange(10USD, :GBP, 0.5) == 5GBP # Specify rate
AirBorne.exchange(10USD, :GBP, 1//2) == 5GBP # Specify rate
AirBorne.exchange(10USD, :USD, exchangeRateI) == 10USDa # Same Currency 
AirBorne.exchange(10USD, :UYU, exchangeRateI) == 500UYU # Known Exchange
AirBorne.exchange(500UYU, :USD, exchangeRateI) == 10USDa # Known Inverse

w0 = Wallet(USD)
w1 = Wallet(:USD)
w0 == w1
w1[:USD] == 0
w1[:GBP] == 0
length(w1) == 1

w2 = Wallet(20USD)
w3 = Wallet(50UYU)
w4 = Wallet(Dict(:USD => 20, :UYU => 50))
w5 = deepcopy(w3)
w6 = Wallet(Dict(:USD => 20, :UYU => 60))
w5[:USD] = 10.0
w2 + 50UYU == w4
20USD + w3 == w4
w2 + w3 == w4
w2 + w4 == Wallet(Dict(:USD => 40, :UYU => 50))
20USD + 50UYU == w4
w5[:USD] == 10
(w4 == w2) == false
(w4 == w6) == false


true

### Data Structures

In [6]:
data

LoadError: UndefVarError: data not defined

In [6]:
using DataStructures: SortedDict

In [7]:
"Data Structure"
struct OHLCV_A
    open::Float64
    high::Float64
    low::Float64
    close::Float64
    volume::Float64    
end

struct OHLCV_B
    ticker::String
    open::Float64
    high::Float64
    low::Float64
    close::Float64
    volume::Float64    
end


const Snapshot_OHLCV_A =  Dict{Symbol, OHLCV_A} # Contains one OHLCV_A entry per ticker

const TimeSeries_OHLCV_A =  SortedDict{Int64, Snapshot_OHLCV_A} # Contains one Snapshot_OHLCV_A  per UNIX timestamp


SortedDict{Int64, Dict{Symbol, OHLCV_A}}

In [19]:
using DataFrames: groupby, DataFrames

g1 = groupby(data2,:unix)
# Multithreading iteration over group # Threads.@threads 
i=0
formatted_data = TimeSeries_OHLCV_A()
a=nothing
b=nothing
for (k, g) in pairs(g1)
    i+=1
    a=k;b=g;
    println(k)
    show(g)
    # Geneate snapshot OHLCV
    
    if i>2
        break
    end
    # do something with `group`
    # formatted_data[k[:unix]]=
end

GroupKey: (unix = 1483453800,)
2×11 SubDataFrame
 Row │ close    high     low      open     volume     date                 uni ⋯
     │ Float64  Float64  Float64  Float64  Int64      DateTime             Int ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 29.0375  29.0825    28.69  28.95    115127600  2017-01-03T10:30:00  148 ⋯
   2 │ 39.307   39.4815    38.79  38.9405   33146000  2017-01-03T10:30:00  148
                                                               5 columns omittedGroupKey: (unix = 1483540200,)
2×11 SubDataFrame
 Row │ close    high     low      open     volume    date                 unix ⋯
     │ Float64  Float64  Float64  Float64  Int64     DateTime             Int6 ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  29.005  29.1275  28.9375  28.9625  84472400  2017-01-04T10:30:00  1483 ⋯
   2 │  39.345  39.567   39.158   39.418   21460000  2017-01-04T10:30:00  1483
                

In [40]:
# a[:unix]
# [ b[1,:symbol]=> i[b,:close] for  i in 1:size(b,1)]
# Dict( [i:symbol=> i for i in b])
# Dict(Pair.(b.symbol, b.close))
using DataFrames: groupby, DataFrame
DataFrame(Dict(pairs(eachcol(b))))


Row,close,currency,date,exchangeName,high,low,open,symbol,timezone,unix,volume
,Float64,String,DateTime,String,Float64,Float64,Float64,String,String,Int64,Int64
1,29.1525,USD,2017-01-05T10:30:00,NMS,29.215,28.9525,28.98,AAPL,America/New_York,1483626600,88774400
2,39.701,USD,2017-01-05T10:30:00,NMS,39.724,39.251,39.304,GOOG,America/New_York,1483626600,26704000


In [79]:
using Parquet2: Dataset
using Tables: Tables
# readdir(joinpath(AirBorne.ETL.Cache.get_cache_path(), AirBorne.ETL.Cache.list_bundles()[1],"archive"))
parquet_path = joinpath(AirBorne.ETL.Cache.get_cache_path(), AirBorne.ETL.Cache.list_bundles()[1],"2023_07_03_11_29_2_972.parq.snappy")
ds =  Dataset(parquet_path)
sch = Tables.schema(ds)
schOG=Tables.schema(data2)

@info sch == schOG

[ Info: true


In [93]:
using Tables: Schema
using Dates: Dates 
using Bijections: Bijection, inv
# sch2 = 
# names(sch)
# using Parquet2: showtree
OHLCV_V1_specs = [
 :close;         Float64;;
 :high;          Float64;;
 :low;           Float64;;
 :open;          Float64;;
 :volume;        Int64;;
 :date;          Dates.DateTime;;
 :unix;          Int64;;
 :exchangeName;  String;;
 :timezone;      String;;
 :currency;      String;;
 :symbol;        String;;
]

specs2 = [
 :exchangeName;  String;;
 :timezone;      String;;
 :currency;      String;;
 :symbol;        String;;
 :close;         Float64;;
 :high;          Float64;;
 :low;           Float64;;
 :open;          Float64;;
 :volume;        Int64;;
 :date;          Dates.DateTime;;
 :unix;          Int64;;
]
sch2 = Schema(OHLCV_V1_specs[1,:],OHLCV_V1_specs[2,:])
sch3 = Schema(specs2[1,:],specs2[2,:]) # Order of columns is important!

@info sch2==sch # Equal
@info sch3==sch # Not equal

schemas = Dict(
    "OHLCV_V1"=>Schema(OHLCV_V1_specs[1,:],OHLCV_V1_specs[2,:])
)
schemas = Dict{Schema,String}(
    Schema(OHLCV_V1_specs[1,:],OHLCV_V1_specs[2,:])=>"OHLCV_V1"
)
schemas = Bijection{String,Schema}()
schemas["OHLCV_V1"] = Schema(OHLCV_V1_specs[1,:],OHLCV_V1_specs[2,:])
@info schemas

[ Info: true
[ Info: false
[ Info: Bijection{String,Schema} (with 1 pairs)


In [94]:
# @info get(schemas,Tables.schema(data2),"UNREGISTERED_SCHEMA")
# @info get(schemas,sch3,"UNREGISTERED_SCHEMA")
schemas(Tables.schema(data2))
inv(schemas)("OHLCV_V1")

Tables.Schema:
 :close         Float64
 :high          Float64
 :low           Float64
 :open          Float64
 :volume        Int64
 :date          Dates.DateTime
 :unix          Int64
 :exchangeName  String
 :timezone      String
 :currency      String
 :symbol        String

In [130]:
module Transform8
export getSchema, addSchema

using Tables: Schema
using Dates: Dates
using Bijections: Bijection, inv

OHLCV_V1_specs = [
    :close;         Float64;;
    :high;          Float64;;
    :low;           Float64;;
    :open;          Float64;;
    :volume;        Int64;;
    :date;          Dates.DateTime;;
    :unix;          Int64;;
    :exchangeName;  String;;
    :timezone;      String;;
    :currency;      String;;
    :symbol;        String;;
   ]

# Better implementation can be achieved by using Schemata.jl (left for ETL V2)
schemas = Bijection{String,Schema}()
getSchema(s::Schema)=get(inv(schemas),s,"UNREGISTERED_SCHEMA")
getSchema(s::String)=get(schemas,s,nothing)
function addSchema(name::String,schema::Schema)
        schemas[name]=schema
end
addSchema("OHLCV_V1",Schema(OHLCV_V1_specs[1,:],OHLCV_V1_specs[2,:]))

end

Main.Transform8

In [165]:
import .Transform8: Transform8
using Tables: schema
T=Transform8
# @info T.getSchema(sch)
# @info T.getSchema("OHLCV_V1")
# @info T.getSchema("OHLCV_V2")
# # T.addSchema("OHLCV_V2",sch3)
# @info T.getSchema("OHLCV_V2")
# @info T.schemas
# get(Transform5.schemas,"OHLCV_V1",nothing)
# get(inv(Transform5.schemas),sch,nothing)


# show(data)
meta=Dict()
meta["schema"]= T.getSchema(schema(data))
AirBorne.ETL.Cache.store_bundle(data;meta=meta, bundle_id="demo", archive=true) 


[ Info: Storing /root/project/.AirBorneCache/demo/2023_07_03_18_29_49_139.parq.snappy


✏ Parquet2.FileWriter{IOStream}(/root/project/.AirBorneCache/demo/2023_07_03_18_29_49_139.parq.snappy)

In [169]:
using AirBorne.ETL.Cache: get_cache_path
using Parquet2: metadata
using Dates: DateTime

get_cache_path()

function describe_bundles(;archive=false)
   cp = get_cache_path()
   bundles = readdir(cp; sort=false)
   l=[]
   # contents_to_read = [c for c in bundles]
   for b=bundles
        contents = readdir(joinpath(cp,b); sort=false) 
       f = [c for c in contents if c != "archive"]
       filename=f[1]
        status="OK"
       ds = Dataset(joinpath(cp,b,filename))
       m=metadata(ds)
       entry = Dict(
            "bundle"=>b,
            "file"=>filename,
            "schema"=>get(m,"schema","MISSING"),
            "storedDate"=>DateTime(filename[1:21], "yyyy_mm_dd_H_M_S_s"),
            "is_archived"=>false,
            "status"=>status,
        )
        push!(l,entry)
        
       @info entry
        if archive
        archive_files = readdir(joinpath(cp,b,"archive"); sort=false)
            for filename=archive_files
               try
               status="OK"
               ds = Dataset(joinpath(cp,b,"archive",filename))
               m=metadata(ds)
                catch e
               status=e.msg
               m=Dict()
               end
               entry = Dict(
                    "bundle"=>b,
                    "file"=>filename,
                    "schema"=>get(m,"schema","MISSING"),
                    "storedDate"=>DateTime(filename[1:21], "yyyy_mm_dd_H_M_S_s"),
                    "is_archived"=>true,
                    "status"=>status,
                )
                push!(l,entry)
                
            end
            
        end
        
   end
    
    return l
end
DataFrame(describe_bundles(archive=true))

[ Info: Dict{String, Any}("status" => "OK", "bundle" => "demo", "file" => "2023_07_03_18_29_49_139.parq.snappy", "is_archived" => false, "schema" => "OHLCV_V1", "storedDate" => DateTime("2023-07-03T18:29:49.100"))


Row,status,bundle,file,is_archived,schema,storedDate
,String,String,String,Bool,String,DateTime
1,OK,demo,2023_07_03_18_29_49_139.parq.snappy,false,OHLCV_V1,2023-07-03T18:29:49.100
2,OK,demo,2023_06_09_19_41_53_457.parq.snappy,true,MISSING,2023-06-09T19:41:53.400
3,OK,demo,2023_06_11_20_26_37_681.parq.snappy,true,MISSING,2023-06-11T20:26:37.600
4,invalid parquet: buffer of length 4 is shorter than minimum size 12,demo,2023_06_22_12_28_57_227.parq.snappy,true,MISSING,2023-06-22T12:28:57.200
5,invalid parquet: buffer of length 4 is shorter than minimum size 12,demo,2023_06_22_12_30_33_667.parq.snappy,true,MISSING,2023-06-22T12:30:33.600
6,OK,demo,2023_06_22_12_30_43_121.parq.snappy,true,MISSING,2023-06-22T12:30:43.100
7,OK,demo,2023_07_02_18_31_35_022.parq.snappy,true,MISSING,2023-07-02T18:31:35
8,OK,demo,2023_07_02_18_32_10_265.parq.snappy,true,MISSING,2023-07-02T18:32:10.200
9,OK,demo,2023_07_02_18_32_49_343.parq.snappy,true,MISSING,2023-07-02T18:32:49.300


In [157]:
a= "2023_07_03_17_1_1_324.parq.snappy"

a[1:21]


2023-07-03T17:01:01.324